# Example Vessel Traffic

Data : Cargo traffic in the gulf of Mexico for year 2019 (https://marinecadastre.gov/)
<br/>
Libraries : Pandas, MovingPandas, transBigData

In [6]:
from shapely.geometry import Polygon
import pandas as pd
import geopandas as gpd
import movingpandas as mpd
import hvplot.pandas
import transbigdata as tbd
from datetime import timedelta
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

Data stored in S3 bucket as parquet format that pandas can read in directly. 
<br/>
**BaseDateTime** : Time of Ais Transmission <br/>
**MMSI** : Unique identifier for ship <br/>
**LAT** : Latitude at BaseDateTime <br/>
**LON** : Longitude at BaseDateTime <br/>
**SOG** : Speed over ground at BaseDateTime <br/>
**COG** : Course over ground at BaseDateTime <br/>
**Heading** : Bearing at BaseDateTime <br/>
**VesselName** : Registered ship name <br/>
**IMO** : International Maritime Organization unique number <br/>
**VesselType** : Vessel class number <br/>
**Draft** : distance between the ship’s keel and the waterline <br/>

In [2]:
dfc = pd.read_parquet("https://pydatatt.s3.amazonaws.com/Combined/GOM_cargo_2019.geoparquet")
dfc = dfc.drop(['t', 'geometry', 'Region'],axis=1).reset_index(drop=True)
dfc

,BaseDateTime,MMSI,LAT,LON,SOG,COG,Heading,VesselName,IMO,VesselType,Draft
0,2019-01-01 00:00:00,314263000.0,29.91651,-90.12085,12.8,150.1,137.0,LYCAVITOS,IMO9368857,70.0,12.8
1,2019-01-01 00:05:00,314263000.0,29.90751,-90.10366,12.5,101.2,97.0,LYCAVITOS,IMO9368857,70.0,12.8
2,2019-01-01 00:10:00,314263000.0,29.91338,-90.08699,12.6,68.7,68.0,LYCAVITOS,IMO9368857,70.0,12.8
3,2019-01-01 00:15:00,314263000.0,29.92024,-90.06981,13.3,67.7,68.0,LYCAVITOS,IMO9368857,70.0,12.8
4,2019-01-01 00:20:00,314263000.0,29.93362,-90.05813,13.2,43.5,37.0,LYCAVITOS,IMO9368857,70.0,12.8
...,...,...,...,...,...,...,...,...,...,...,...
8469578,2019-12-31 23:15:00,367141680.0,28.74632,-93.68953,8.6,308.0,307.0,NATIONAL GLORY,IMO8302246,70.0,9.1
8469579,2019-12-31 23:35:00,367141680.0,28.77250,-93.73695,8.6,304.4,302.0,NATIONAL GLORY,IMO8302246,70.0,9.1
8469580,2019-12-31 23:45:00,367141680.0,28.78780,-93.76314,8.6,303.9,303.0,NATIONAL GLORY,IMO8302246,70.0,9.1
8469581,2019-12-31 23:50:00,367141680.0,28.79382,-93.76986,8.6,304.2,303.0,NATIONAL GLORY,IMO8302246,70.0,9.1


In [5]:
dft = pd.read_parquet("https://pydatatt.s3.amazonaws.com/Combined/GOM_tanker_2019.geoparquet")
dft = dft.drop(['t', 'geometry', 'Region'],axis=1).reset_index(drop=True)
dft

,BaseDateTime,MMSI,LAT,LON,SOG,COG,Heading,VesselName,IMO,VesselType,Draft
0,2019-01-01 00:00:00,403638000.0,28.52015,-94.34360,0.1,128.8,76.0,GHINAH,IMO9484728,80.0,22.5
1,2019-01-01 00:05:00,403638000.0,28.52016,-94.34354,0.1,161.3,72.0,GHINAH,IMO9484728,80.0,22.5
2,2019-01-01 00:10:00,403638000.0,28.52004,-94.34356,0.0,179.7,71.0,GHINAH,IMO9484728,80.0,22.5
3,2019-01-01 00:15:00,403638000.0,28.52007,-94.34355,0.1,174.4,71.0,GHINAH,IMO9484728,80.0,22.5
4,2019-01-01 00:20:00,403638000.0,28.51993,-94.34365,0.1,247.8,69.0,GHINAH,IMO9484728,80.0,22.5
...,...,...,...,...,...,...,...,...,...,...,...
12330670,2019-12-31 23:35:00,248993000.0,29.19772,-94.43065,10.3,309.0,310.0,MINERVA ALICE,IMO9309435,80.0,11.4
12330671,2019-12-31 23:40:00,248993000.0,29.20818,-94.44423,9.8,314.0,315.0,MINERVA ALICE,IMO9309435,80.0,11.4
12330672,2019-12-31 23:45:00,248993000.0,29.21473,-94.45425,8.4,315.0,315.0,MINERVA ALICE,IMO9309435,80.0,11.4
12330673,2019-12-31 23:50:00,248993000.0,29.22153,-94.46270,6.3,342.0,346.0,MINERVA ALICE,IMO9309435,80.0,11.4


In [7]:
dfc['Date'] = dfc.BaseDateTime.apply(lambda x: datetime(x.year,x.month,x.day))
dft['Date'] = dft.BaseDateTime.apply(lambda x: datetime(x.year,x.month,x.day))

In [30]:
dft.groupby('Date').VesselName.nunique().rename('Tanker').hvplot(title="Daily GOM traffic 2019", height=500, ylabel="Vessels/Day") *\
dfc.groupby('Date').VesselName.nunique().rename('Cargo').hvplot(title="Daily GOM traffic 2019", height=500)

:Overlay
   .Curve.Tanker :Curve   [Date]   (Tanker)
   .Curve.Cargo  :Curve   [Date]   (Cargo)